In [38]:
import glob
import random
import os
import numpy as np
import torch

from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms

# Normalization parameters for pre-trained PyTorch models
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])


class Dataset(Dataset):
    def __init__(self, dataset_path, split_path, split_number, input_shape, sequence_length, training):
        self.training = training
        self.label_index = self._extract_label_mapping(split_path)
        self.sequences = self._extract_sequence_paths(dataset_path, split_path, split_number, training)
        self.sequence_length = sequence_length
        self.label_names = sorted(list(set([self._activity_from_path(seq_path) for seq_path in self.sequences])))
        self.num_classes = len(self.label_names)
        self.transform = transforms.Compose(
            [
                transforms.Resize(input_shape[-2:], Image.BICUBIC),
                transforms.ToTensor(),
                transforms.Normalize(mean, std),
            ]
        )

    def _extract_label_mapping(self, split_path="data/ucfTrainTestlist"):
        """ Extracts a mapping between activity name and softmax index """
        with open(os.path.join(split_path, "classInd.txt")) as file:
            lines = file.read().splitlines()
        label_mapping = {}
        for line in lines:
            label, action = line.split()
            label_mapping[action] = int(label) - 1
        return label_mapping

    def _extract_sequence_paths(
        self, dataset_path, split_path="data/ucfTrainTestlist", split_number=1, training=True
    ):
        """ Extracts paths to sequences given the specified train / test split """
        assert split_number in [1, 2, 3], "Split number has to be one of {1, 2, 3}"
        fn = f"trainlist0{split_number}.txt" if training else f"testlist0{split_number}.txt"
        split_path = os.path.join(split_path, fn)
        with open(split_path) as file:
            lines = file.read().splitlines()
        sequence_paths = []
        for line in lines:
            seq_name = line.split(".avi")[0]
            sequence_paths += [os.path.join(dataset_path, seq_name)]
        return sequence_paths

    def _activity_from_path(self, path):
        """ Extracts activity name from filepath """
        return path.split("/")[-2]

    def _frame_number(self, image_path):
        """ Extracts frame number from filepath """
        return int(image_path.split("/")[-1].split(".jpg")[0])

    def _pad_to_length(self, sequence):
        """ Pads the sequence to required sequence length """
        left_pad = sequence[0]
        if self.sequence_length is not None:
            while len(sequence) < self.sequence_length:
                sequence.insert(0, left_pad)
        return sequence

    def __getitem__(self, index):
        sequence_path = self.sequences[index % len(self)]
        # Sort frame sequence based on frame number
        image_paths = sorted(glob.glob(f"{sequence_path}/*.jpg"), key=lambda path: self._frame_number(path))
        # Pad frames sequences shorter than `self.sequence_length` to length
        print (index, sequence_path)
        image_paths = self._pad_to_length(image_paths)
        if self.training:
            # Randomly choose sample interval and start frame
            sample_interval = np.random.randint(1, len(image_paths) // self.sequence_length + 1)
            start_i = np.random.randint(0, len(image_paths) - sample_interval * self.sequence_length + 1)
            flip = np.random.random() < 0.5
        else:
            # Start at first frame and sample uniformly over sequence
            start_i = 0
            sample_interval = 1 if self.sequence_length is None else len(image_paths) // self.sequence_length
            flip = False
        # Extract frames as tensors
        image_sequence = []
        for i in range(start_i, len(image_paths), sample_interval):
            if self.sequence_length is None or len(image_sequence) < self.sequence_length:
                image_tensor = self.transform(Image.open(image_paths[i]))
                if flip:
                    image_tensor = torch.flip(image_tensor, (-1,))
                image_sequence.append(image_tensor)
        image_sequence = torch.stack(image_sequence)
        target = self.label_index[self._activity_from_path(sequence_path)]
        return image_sequence, target

    def __len__(self):
        return len(self.sequences)


In [39]:
dataset_path = 'data/UCF-101-frames'
split_path = 'data/ucfTrainTestlist'
split_number = 1
image_shape = (3, 112, 112)
sequence_length = 40

train_dataset = Dataset(
    dataset_path=dataset_path,
    split_path=split_path,
    split_number=split_number,
    input_shape=image_shape,
    sequence_length=sequence_length,
    training=True,
)

test_dataset = Dataset(
    dataset_path=dataset_path,
    split_path=split_path,
    split_number=split_number,
    input_shape=image_shape,
    sequence_length=sequence_length,
    training=False,
)

In [31]:
import torch
import sys
import numpy as np
import itertools
from models import *
from dataset import *
from torch.utils.data import DataLoader
from torch.autograd import Variable
import argparse
import time
import datetime

In [40]:
batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=1)

In [42]:
for i, (frames, labels) in enumerate(test_dataloader):
    print ('iter', i)
#     break

0 data/UCF-101-frames/ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c01
1 data/UCF-101-frames/ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c02
2 data/UCF-101-frames/ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c03
3 data/UCF-101-frames/ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c04
4 data/UCF-101-frames/ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c05
5 data/UCF-101-frames/ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c06
6 data/UCF-101-frames/ApplyEyeMakeup/v_ApplyEyeMakeup_g02_c01
7 data/UCF-101-frames/ApplyEyeMakeup/v_ApplyEyeMakeup_g02_c02
8 data/UCF-101-frames/ApplyEyeMakeup/v_ApplyEyeMakeup_g02_c03
9 data/UCF-101-frames/ApplyEyeMakeup/v_ApplyEyeMakeup_g02_c04
10 data/UCF-101-frames/ApplyEyeMakeup/v_ApplyEyeMakeup_g03_c01
11 data/UCF-101-frames/ApplyEyeMakeup/v_ApplyEyeMakeup_g03_c02
12 data/UCF-101-frames/ApplyEyeMakeup/v_ApplyEyeMakeup_g03_c03
13 data/UCF-101-frames/ApplyEyeMakeup/v_ApplyEyeMakeup_g03_c04
14 data/UCF-101-frames/ApplyEyeMakeup/v_ApplyEyeMakeup_g03_c05
15 data/UCF-101-frames/ApplyEyeMakeup/v_ApplyEyeMakeup_g03_c06
16

140 data/UCF-101-frames/BabyCrawling/v_BabyCrawling_g06_c01
141 data/UCF-101-frames/BabyCrawling/v_BabyCrawling_g06_c02
142 data/UCF-101-frames/BabyCrawling/v_BabyCrawling_g06_c03
143 data/UCF-101-frames/BabyCrawling/v_BabyCrawling_g06_c04
144 
data/UCF-101-frames/BabyCrawling/v_BabyCrawling_g06_c05145 data/UCF-101-frames/BabyCrawling/v_BabyCrawling_g06_c06
146 data/UCF-101-frames/BabyCrawling/v_BabyCrawling_g07_c01
iter 8
147 data/UCF-101-frames/BabyCrawling/v_BabyCrawling_g07_c02
148 data/UCF-101-frames/BabyCrawling/v_BabyCrawling_g07_c03
149 data/UCF-101-frames/BabyCrawling/v_BabyCrawling_g07_c04
150 data/UCF-101-frames/BabyCrawling/v_BabyCrawling_g07_c05
151 data/UCF-101-frames/BabyCrawling/v_BabyCrawling_g07_c06
152 data/UCF-101-frames/BalanceBeam/v_BalanceBeam_g01_c01
153 data/UCF-101-frames/BalanceBeam/v_BalanceBeam_g01_c02
154 data/UCF-101-frames/BalanceBeam/v_BalanceBeam_g01_c03
155 data/UCF-101-frames/BalanceBeam/v_BalanceBeam_g01_c04
156 data/UCF-101-frames/BalanceBeam/v_Bal

275 data/UCF-101-frames/Basketball/v_Basketball_g01_c07
276 data/UCF-101-frames/Basketball/v_Basketball_g02_c01
277 data/UCF-101-frames/Basketball/v_Basketball_g02_c02
278 data/UCF-101-frames/Basketball/v_Basketball_g02_c03
279 data/UCF-101-frames/Basketball/v_Basketball_g02_c04
280 data/UCF-101-frames/Basketball/v_Basketball_g02_c05
281 data/UCF-101-frames/Basketball/v_Basketball_g02_c06
282 data/UCF-101-frames/Basketball/v_Basketball_g03_c01
283 data/UCF-101-frames/Basketball/v_Basketball_g03_c02
284 data/UCF-101-frames/Basketball/v_Basketball_g03_c03
285 data/UCF-101-frames/Basketball/v_Basketball_g03_c04
286 data/UCF-101-frames/Basketball/v_Basketball_g03_c05
287 data/UCF-101-frames/Basketball/v_Basketball_g03_c06
288 data/UCF-101-frames/Basketball/v_Basketball_g04_c01
289 data/UCF-101-frames/Basketball/v_Basketball_g04_c02
290 data/UCF-101-frames/Basketball/v_Basketball_g04_c03
iter 17
291 data/UCF-101-frames/Basketball/v_Basketball_g04_c04
292 data/UCF-101-frames/Basketball/v_Bas

419 data/UCF-101-frames/Biking/v_Biking_g06_c04
420 data/UCF-101-frames/Biking/v_Biking_g06_c05
421 data/UCF-101-frames/Biking/v_Biking_g07_c01
422 data/UCF-101-frames/Biking/v_Biking_g07_c02
423 data/UCF-101-frames/Biking/v_Biking_g07_c03
424 data/UCF-101-frames/Biking/v_Biking_g07_c04
425 data/UCF-101-frames/Biking/v_Biking_g07_c05
426 data/UCF-101-frames/Biking/v_Biking_g07_c06
427 data/UCF-101-frames/Billiards/v_Billiards_g01_c01
428 data/UCF-101-frames/Billiards/v_Billiards_g01_c02
429 data/UCF-101-frames/Billiards/v_Billiards_g01_c03
430 data/UCF-101-frames/Billiards/v_Billiards_g01_c04
431 data/UCF-101-frames/Billiards/v_Billiards_g01_c05
432 data/UCF-101-frames/Billiards/v_Billiards_g01_c06
433 data/UCF-101-frames/Billiards/v_Billiards_g02_c01
434 data/UCF-101-frames/Billiards/v_Billiards_g02_c02
iter 26
435 data/UCF-101-frames/Billiards/v_Billiards_g02_c03
436 data/UCF-101-frames/Billiards/v_Billiards_g02_c04
437 data/UCF-101-frames/Billiards/v_Billiards_g02_c05
438 data/UCF-1

556 data/UCF-101-frames/BodyWeightSquats/v_BodyWeightSquats_g05_c02
557 data/UCF-101-frames/BodyWeightSquats/v_BodyWeightSquats_g05_c03
558 data/UCF-101-frames/BodyWeightSquats/v_BodyWeightSquats_g05_c04
559 data/UCF-101-frames/BodyWeightSquats/v_BodyWeightSquats_g06_c01
560 data/UCF-101-frames/BodyWeightSquats/v_BodyWeightSquats_g06_c02
561 data/UCF-101-frames/BodyWeightSquats/v_BodyWeightSquats_g06_c03
562 data/UCF-101-frames/BodyWeightSquats/v_BodyWeightSquats_g06_c04
iter 34
563 data/UCF-101-frames/BodyWeightSquats/v_BodyWeightSquats_g06_c05
564 data/UCF-101-frames/BodyWeightSquats/v_BodyWeightSquats_g07_c01
565 data/UCF-101-frames/BodyWeightSquats/v_BodyWeightSquats_g07_c02
566 data/UCF-101-frames/BodyWeightSquats/v_BodyWeightSquats_g07_c03
567 data/UCF-101-frames/BodyWeightSquats/v_BodyWeightSquats_g07_c04
568 data/UCF-101-frames/Bowling/v_Bowling_g01_c01
569 data/UCF-101-frames/Bowling/v_Bowling_g01_c02
570 data/UCF-101-frames/Bowling/v_Bowling_g01_c03
571 data/UCF-101-frames/Bo

...

1375 data/UCF-101-frames/HandstandPushups/v_HandStandPushups_g01_c01
1376 data/UCF-101-frames/HandstandPushups/v_HandStandPushups_g01_c02
1377 data/UCF-101-frames/HandstandPushups/v_HandStandPushups_g01_c03
1378 data/UCF-101-frames/HandstandPushups/v_HandStandPushups_g01_c04
iter 85
1379 data/UCF-101-frames/HandstandPushups/v_HandStandPushups_g02_c01
1380 data/UCF-101-frames/HandstandPushups/v_HandStandPushups_g02_c02
1381 data/UCF-101-frames/HandstandPushups/v_HandStandPushups_g02_c03
1382 data/UCF-101-frames/HandstandPushups/v_HandStandPushups_g02_c04
1383 data/UCF-101-frames/HandstandPushups/v_HandStandPushups_g03_c01
1384 data/UCF-101-frames/HandstandPushups/v_HandStandPushups_g03_c02
1385 data/UCF-101-frames/HandstandPushups/v_HandStandPushups_g03_c03
1386 data/UCF-101-frames/HandstandPushups/v_HandStandPushups_g03_c04
1387 data/UCF-101-frames/HandstandPushups/v_HandStandPushups_g04_c01
1388 data/UCF-101-frames/HandstandPushups/v_HandStandPushups_g04_c02
1389 data/UCF-101-frames/H

1505 data/UCF-101-frames/HighJump/v_HighJump_g06_c01
1506 data/UCF-101-frames/HighJump/v_HighJump_g06_c02
iter 93
1507 data/UCF-101-frames/HighJump/v_HighJump_g06_c03
1508 data/UCF-101-frames/HighJump/v_HighJump_g06_c04
1509 data/UCF-101-frames/HighJump/v_HighJump_g07_c01
1510 data/UCF-101-frames/HighJump/v_HighJump_g07_c02
1511 data/UCF-101-frames/HighJump/v_HighJump_g07_c03
1512 data/UCF-101-frames/HighJump/v_HighJump_g07_c04
1513 data/UCF-101-frames/HighJump/v_HighJump_g07_c05
1514 data/UCF-101-frames/HighJump/v_HighJump_g07_c06
1515 data/UCF-101-frames/HorseRace/v_HorseRace_g01_c01
1516 data/UCF-101-frames/HorseRace/v_HorseRace_g01_c02
1517 data/UCF-101-frames/HorseRace/v_HorseRace_g01_c03
1518 data/UCF-101-frames/HorseRace/v_HorseRace_g01_c04
1519 data/UCF-101-frames/HorseRace/v_HorseRace_g02_c01
1520 data/UCF-101-frames/HorseRace/v_HorseRace_g02_c02
1521 data/UCF-101-frames/HorseRace/v_HorseRace_g02_c03
1522 data/UCF-101-frames/HorseRace/v_HorseRace_g02_c04
iter 94
1523 data/UCF-

1650 data/UCF-101-frames/IceDancing/v_IceDancing_g03_c04
iter 102
1651 data/UCF-101-frames/IceDancing/v_IceDancing_g03_c05
1652 data/UCF-101-frames/IceDancing/v_IceDancing_g03_c06
1653 data/UCF-101-frames/IceDancing/v_IceDancing_g04_c01
1654 data/UCF-101-frames/IceDancing/v_IceDancing_g04_c02
1655 data/UCF-101-frames/IceDancing/v_IceDancing_g04_c03
1656 data/UCF-101-frames/IceDancing/v_IceDancing_g04_c04
1657 data/UCF-101-frames/IceDancing/v_IceDancing_g04_c05
1658 data/UCF-101-frames/IceDancing/v_IceDancing_g04_c06
1659 data/UCF-101-frames/IceDancing/v_IceDancing_g04_c07
1660 data/UCF-101-frames/IceDancing/v_IceDancing_g05_c01
1661 data/UCF-101-frames/IceDancing/v_IceDancing_g05_c02
1662 data/UCF-101-frames/IceDancing/v_IceDancing_g05_c03
1663 data/UCF-101-frames/IceDancing/v_IceDancing_g05_c04
1664 data/UCF-101-frames/IceDancing/v_IceDancing_g05_c05
1665 data/UCF-101-frames/IceDancing/v_IceDancing_g05_c06
1666 data/UCF-101-frames/IceDancing/v_IceDancing_g06_c01
iter 103
1667 data/UCF

IndexError: Caught IndexError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/sangwooji/miniconda3/envs/colorfusion/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 202, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/sangwooji/miniconda3/envs/colorfusion/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/sangwooji/miniconda3/envs/colorfusion/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/tmp/ipykernel_31263/201948710.py", line 79, in __getitem__
    image_paths = self._pad_to_length(image_paths)
  File "/tmp/ipykernel_31263/201948710.py", line 67, in _pad_to_length
    left_pad = sequence[0]
IndexError: list index out of range



1713 data/UCF-101-frames/JugglingBalls/v_JugglingBalls_g01_c04
1714 data/UCF-101-frames/JugglingBalls/v_JugglingBalls_g02_c01
1715 data/UCF-101-frames/JugglingBalls/v_JugglingBalls_g02_c02
1716 data/UCF-101-frames/JugglingBalls/v_JugglingBalls_g02_c03
1717 data/UCF-101-frames/JugglingBalls/v_JugglingBalls_g02_c04
1718 data/UCF-101-frames/JugglingBalls/v_JugglingBalls_g02_c05
1719 data/UCF-101-frames/JugglingBalls/v_JugglingBalls_g02_c06
1720 data/UCF-101-frames/JugglingBalls/v_JugglingBalls_g03_c01
1721 data/UCF-101-frames/JugglingBalls/v_JugglingBalls_g03_c02
1722 data/UCF-101-frames/JugglingBalls/v_JugglingBalls_g03_c03
1723 data/UCF-101-frames/JugglingBalls/v_JugglingBalls_g03_c04
1724 data/UCF-101-frames/JugglingBalls/v_JugglingBalls_g03_c05
1725 data/UCF-101-frames/JugglingBalls/v_JugglingBalls_g03_c06
1726 data/UCF-101-frames/JugglingBalls/v_JugglingBalls_g03_c07
1727 data/UCF-101-frames/JugglingBalls/v_JugglingBalls_g04_c01
1728 data/UCF-101-frames/JugglingBalls/v_JugglingBalls

In [52]:
def get_temp(idx):
    sequence_path = test_dataset.sequences[idx]
    print (sequence_path)
    image_paths = sorted(glob.glob(f"{sequence_path}/*.jpg"), key=lambda path: test_dataset._frame_number(path))
    image_paths = test_dataset._pad_to_length(image_paths)
    print (image_paths)
    
for i in range(1700, 1750):
    get_temp(i)

data/UCF-101-frames/JavelinThrow/v_JavelinThrow_g05_c06
['data/UCF-101-frames/JavelinThrow/v_JavelinThrow_g05_c06/0.jpg', 'data/UCF-101-frames/JavelinThrow/v_JavelinThrow_g05_c06/1.jpg', 'data/UCF-101-frames/JavelinThrow/v_JavelinThrow_g05_c06/2.jpg', 'data/UCF-101-frames/JavelinThrow/v_JavelinThrow_g05_c06/3.jpg', 'data/UCF-101-frames/JavelinThrow/v_JavelinThrow_g05_c06/4.jpg', 'data/UCF-101-frames/JavelinThrow/v_JavelinThrow_g05_c06/5.jpg', 'data/UCF-101-frames/JavelinThrow/v_JavelinThrow_g05_c06/6.jpg', 'data/UCF-101-frames/JavelinThrow/v_JavelinThrow_g05_c06/7.jpg', 'data/UCF-101-frames/JavelinThrow/v_JavelinThrow_g05_c06/8.jpg', 'data/UCF-101-frames/JavelinThrow/v_JavelinThrow_g05_c06/9.jpg', 'data/UCF-101-frames/JavelinThrow/v_JavelinThrow_g05_c06/10.jpg', 'data/UCF-101-frames/JavelinThrow/v_JavelinThrow_g05_c06/11.jpg', 'data/UCF-101-frames/JavelinThrow/v_JavelinThrow_g05_c06/12.jpg', 'data/UCF-101-frames/JavelinThrow/v_JavelinThrow_g05_c06/13.jpg', 'data/UCF-101-frames/JavelinT

IndexError: list index out of range

In [ ]:

sequence_path = self.sequences[index % len(self)]
# Sort frame sequence based on frame number
image_paths = sorted(glob.glob(f"{sequence_path}/*.jpg"), key=lambda path: self._frame_number(path))
# Pad frames sequences shorter than `self.sequence_length` to length
print (index, sequence_path)
image_paths = self._pad_to_length(image_paths)